In [21]:
import os
import glob
import yaml
import tqdm.auto as tqdm

# Load in template

In [22]:
path_template_config = "/cellar/users/aklie/opt/gene_program_evaluation/examples/evaluation/iPSC_EC/evaluation_pipeline.yml"

In [23]:
# load template config
with open(path_template_config, 'r') as f:
    template_config = yaml.safe_load(f)
template_config

{'io': {'path_mdata': '/cellar/users/aklie/opt/gene_program_evaluation/examples/inference/iPSC_EC/cNMF/cNMF_30_0.2_gene_names.h5mu',
  'path_out': '/cellar/users/aklie/opt/gene_program_evaluation/examples/evaluation/iPSC_EC/cNMF/cNMF_30',
  'data_key': 'rna',
  'prog_key': 'cNMF'},
 'categorical_association': {'categorical_keys': ['sample'],
  'pseudobulk_key': None,
  'test': 'pearsonr',
  'mode': 'one_vs_all',
  'n_jobs': -1,
  'inplace': False},
 'perturbation_association': {'groupby_key': 'sample',
  'collapse_targets': True,
  'pseudobulk': False,
  'reference_targets': ['non-targeting'],
  'n_jobs': -1,
  'inplace': False},
 'gene_set_enrichment': {'prog_nam': None,
  'organism': 'human',
  'libraries': ['Reactome_2022', 'GO_Biological_Process_2023'],
  'method': 'fisher',
  'database': 'enrichr',
  'n_top': 500,
  'low_cutoff': 0.0,
  'n_jobs': -1,
  'inplace': False,
  'user_geneset': None,
  'max_size': 500,
  'min_size': 5},
 'trait_enrichment': {'gwas_data': '/cellar/users/a

# Identify all possible MuData's to be evaluated

In [24]:
path_inference = "/cellar/users/aklie/opt/gene_program_evaluation/examples/inference/iPSC_EC"
path_out = "/cellar/users/aklie/opt/gene_program_evaluation/examples/evaluation/iPSC_EC"

In [25]:
path_h5mus = glob.glob(os.path.join(path_inference, "*", "*.h5mu"))
path_h5mus

['/cellar/users/aklie/opt/gene_program_evaluation/examples/inference/iPSC_EC/cNMF/cNMF_200_0.2_gene_names.h5mu',
 '/cellar/users/aklie/opt/gene_program_evaluation/examples/inference/iPSC_EC/cNMF/cNMF_80_0.2_gene_names.h5mu',
 '/cellar/users/aklie/opt/gene_program_evaluation/examples/inference/iPSC_EC/cNMF/cNMF_30_0.2_gene_names.h5mu',
 '/cellar/users/aklie/opt/gene_program_evaluation/examples/inference/iPSC_EC/cNMF/cNMF_300_0.2_gene_names.h5mu',
 '/cellar/users/aklie/opt/gene_program_evaluation/examples/inference/iPSC_EC/cNMF/cNMF_60_0.2_gene_names.h5mu',
 '/cellar/users/aklie/opt/gene_program_evaluation/examples/inference/iPSC_EC/cNMF/cNMF_100_0.2_gene_names.h5mu',
 '/cellar/users/aklie/opt/gene_program_evaluation/examples/inference/iPSC_EC/cNMF/cNMF_250_0.2_gene_names.h5mu',
 '/cellar/users/aklie/opt/gene_program_evaluation/examples/inference/iPSC_EC/Topyfic/Topyfic_iPSC_EC_13.h5mu',
 '/cellar/users/aklie/opt/gene_program_evaluation/examples/inference/iPSC_EC/Topyfic/Topyfic_iPSC_EC_

In [32]:
basenames = [os.path.basename(p) for p in path_h5mus]
ks = [b.replace("_0.2_gene_names.h5mu", "").replace("iPSC_EC_", "").replace(".h5mu", "").split("_")[1] for b in basenames]
methods = [b.split("_")[0] for b in basenames]
ks[-1] = "100"
methods[-1] = "factor_analysis"
ks, methods

(['200',
  '80',
  '30',
  '300',
  '60',
  '100',
  '250',
  '13',
  '40',
  '17',
  '50',
  '12',
  '16',
  '45',
  '18',
  '35',
  '10',
  '14',
  '25',
  '20',
  '11',
  '15',
  '30',
  '19',
  '5',
  '100'],
 ['cNMF',
  'cNMF',
  'cNMF',
  'cNMF',
  'cNMF',
  'cNMF',
  'cNMF',
  'Topyfic',
  'Topyfic',
  'Topyfic',
  'Topyfic',
  'Topyfic',
  'Topyfic',
  'Topyfic',
  'Topyfic',
  'Topyfic',
  'Topyfic',
  'Topyfic',
  'Topyfic',
  'Topyfic',
  'Topyfic',
  'Topyfic',
  'Topyfic',
  'Topyfic',
  'Topyfic',
  'factor_analysis'])

In [33]:
# Create path_outs using path_out as base, then adding method dir and finall method_k dir
path_outs = [os.path.join(path_out, method, f"{method}_{k}") for method, k in zip(methods, ks)]
path_outs

['/cellar/users/aklie/opt/gene_program_evaluation/examples/evaluation/iPSC_EC/cNMF/cNMF_200',
 '/cellar/users/aklie/opt/gene_program_evaluation/examples/evaluation/iPSC_EC/cNMF/cNMF_80',
 '/cellar/users/aklie/opt/gene_program_evaluation/examples/evaluation/iPSC_EC/cNMF/cNMF_30',
 '/cellar/users/aklie/opt/gene_program_evaluation/examples/evaluation/iPSC_EC/cNMF/cNMF_300',
 '/cellar/users/aklie/opt/gene_program_evaluation/examples/evaluation/iPSC_EC/cNMF/cNMF_60',
 '/cellar/users/aklie/opt/gene_program_evaluation/examples/evaluation/iPSC_EC/cNMF/cNMF_100',
 '/cellar/users/aklie/opt/gene_program_evaluation/examples/evaluation/iPSC_EC/cNMF/cNMF_250',
 '/cellar/users/aklie/opt/gene_program_evaluation/examples/evaluation/iPSC_EC/Topyfic/Topyfic_13',
 '/cellar/users/aklie/opt/gene_program_evaluation/examples/evaluation/iPSC_EC/Topyfic/Topyfic_40',
 '/cellar/users/aklie/opt/gene_program_evaluation/examples/evaluation/iPSC_EC/Topyfic/Topyfic_17',
 '/cellar/users/aklie/opt/gene_program_evaluatio

In [34]:
# Grab prog_keys from path_outs as the basename of the path_outs
prog_keys = [os.path.basename(path_out) for path_out in path_outs]
prog_keys = [prog_key.split("_")[0] if "cNMF" in prog_key else prog_key for prog_key in prog_keys]
prog_keys = [prog_key.split("_")[0] if "Topyfic" in prog_key else prog_key for prog_key in prog_keys]
prog_keys

['cNMF',
 'cNMF',
 'cNMF',
 'cNMF',
 'cNMF',
 'cNMF',
 'cNMF',
 'Topyfic',
 'Topyfic',
 'Topyfic',
 'Topyfic',
 'Topyfic',
 'Topyfic',
 'Topyfic',
 'Topyfic',
 'Topyfic',
 'Topyfic',
 'Topyfic',
 'Topyfic',
 'Topyfic',
 'Topyfic',
 'Topyfic',
 'Topyfic',
 'Topyfic',
 'Topyfic',
 'factor_analysis_100']

In [35]:
# Write out config files where each config file has a different path_mdata, path_out and prog_key
path_configs = []
for path_h5mu, path_out, prog_key in tqdm.tqdm(zip(path_h5mus, path_outs, prog_keys), total=len(path_h5mus)):
    config = template_config.copy()
    config["io"]["path_mdata"] = path_h5mu
    config["io"]["path_out"] = path_out
    config["io"]["prog_key"] = prog_key

    path_config = os.path.join(path_out, "evaluation_pipeline.yml")
    os.makedirs(path_out, exist_ok=True)
    with open(path_config, 'w') as f:
        yaml.dump(config, f, sort_keys=False)
    print(f"Wrote {path_config}")
    path_configs.append(path_config)

100%|██████████| 26/26 [00:00<00:00, 126.51it/s]

Wrote /cellar/users/aklie/opt/gene_program_evaluation/examples/evaluation/iPSC_EC/cNMF/cNMF_200/evaluation_pipeline.yml
Wrote /cellar/users/aklie/opt/gene_program_evaluation/examples/evaluation/iPSC_EC/cNMF/cNMF_80/evaluation_pipeline.yml
Wrote /cellar/users/aklie/opt/gene_program_evaluation/examples/evaluation/iPSC_EC/cNMF/cNMF_30/evaluation_pipeline.yml
Wrote /cellar/users/aklie/opt/gene_program_evaluation/examples/evaluation/iPSC_EC/cNMF/cNMF_300/evaluation_pipeline.yml
Wrote /cellar/users/aklie/opt/gene_program_evaluation/examples/evaluation/iPSC_EC/cNMF/cNMF_60/evaluation_pipeline.yml
Wrote /cellar/users/aklie/opt/gene_program_evaluation/examples/evaluation/iPSC_EC/cNMF/cNMF_100/evaluation_pipeline.yml
Wrote /cellar/users/aklie/opt/gene_program_evaluation/examples/evaluation/iPSC_EC/cNMF/cNMF_250/evaluation_pipeline.yml
Wrote /cellar/users/aklie/opt/gene_program_evaluation/examples/evaluation/iPSC_EC/Topyfic/Topyfic_13/evaluation_pipeline.yml
Wrote /cellar/users/aklie/opt/gene_pro

In [36]:
# Write a SLURM array script to run all the configs in parallel in the following format
path_script = "/cellar/users/aklie/opt/gene_program_evaluation/examples/evaluation/evaluation_pipeline.py"
path_env = "/cellar/users/aklie/opt/miniconda3/envs/test_celloracle"

script_content = f"""#!/bin/bash
#SBATCH --partition=carter-compute
#SBATCH --job-name=evaluation_pipelines
#SBATCH --output=logs/evaluation_pipelines_%A_%a.out
#SBATCH --time=24:00:00
#SBATCH --mem=16G
#SBATCH --cpus-per-task=4
#SBATCH --array=1-{len(path_configs)}

#####
# USAGE:
# sbatch evaluation_pipelines.sh <config>
#####

# Date
date
echo -e "Job ID: $SLURM_JOB_ID"

# Configuring env
source activate {path_env}

# Input configs should be one per line with tab indentation
configs=(
{os.linesep.join(path_configs)}
)
config=${{configs[$SLURM_ARRAY_TASK_ID-1]}}

# Run the command
echo "Running command: python {path_script} --config $config"
python {path_script} --config $config

# Date
date
"""

path_slurm_out = "/cellar/users/aklie/opt/gene_program_evaluation/examples/evaluation/iPSC_EC"
path_slurm_script = os.path.join(path_slurm_out, "evaluation_pipelines.sh")
with open(path_slurm_script, 'w') as f:
    f.write(script_content)

# DONE!

---